#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [27]:
model = Sequential()

In [28]:
#卷積層1

In [29]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [30]:
model.add(Dropout(rate=0.25))

In [31]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [32]:
#卷積層2與池化層2

In [33]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [34]:
model.add(Dropout(0.25))

In [35]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [36]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [37]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [38]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [39]:
model.add(Dense(10, activation='softmax'))

In [40]:
#檢查model 的STACK
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)              0         
__________

# 載入之前訓練的模型

In [41]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''
model.fit(x_img_train_normalize, y_label_train_OneHot, validation_split = 0.25, epochs = 12, batch_size = 128, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 112s 3ms/step - loss: 2.1651 - acc: 0.1982 - val_loss: 2.0522 - val_acc: 0.2960
Epoch 2/12
37500/37500 [==============================] - 87s 2ms/step - loss: 1.9548 - acc: 0.2961 - val_loss: 1.9350 - val_acc: 0.3461
Epoch 3/12
37500/37500 [==============================] - 92s 2ms/step - loss: 1.8336 - acc: 0.3473 - val_loss: 1.8411 - val_acc: 0.3829
Epoch 4/12
37500/37500 [==============================] - 99s 3ms/step - loss: 1.7302 - acc: 0.3819 - val_loss: 1.7485 - val_acc: 0.4175
Epoch 5/12
37500/37500 [==============================] - 101s 3ms/step - loss: 1.6490 - acc: 0.4104 - val_loss: 1.6802 - val_acc: 0.4399
Epoch 6/12
37500/37500 [==============================] - 98s 3ms/step - loss: 1.5852 - acc: 0.4309 - val_loss: 1.6507 - val_acc: 0.4505
Epoch 7/12
37500/37500 [==============================] - 90s 2ms/step - loss:

In [23]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

10000/10000 [==============================] - 5s 522us/step
accuracy= 0.5453


In [24]:
model.compile(loss='MSE', optimizer='sgd', metrics=['accuracy'])
train_history = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_split = 0.25, epochs = 12, batch_size = 128, verbose = 1)
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 105s 3ms/step - loss: 0.0606 - acc: 0.5310 - val_loss: 0.0629 - val_acc: 0.5490
Epoch 2/12
37500/37500 [==============================] - 96s 3ms/step - loss: 0.0604 - acc: 0.5327 - val_loss: 0.0627 - val_acc: 0.5514
Epoch 3/12
37500/37500 [==============================] - 96s 3ms/step - loss: 0.0604 - acc: 0.5328 - val_loss: 0.0626 - val_acc: 0.5516
Epoch 4/12
37500/37500 [==============================] - 89s 2ms/step - loss: 0.0602 - acc: 0.5368 - val_loss: 0.0627 - val_acc: 0.5502
Epoch 5/12
37500/37500 [==============================] - 86s 2ms/step - loss: 0.0601 - acc: 0.5349 - val_loss: 0.0625 - val_acc: 0.5528
Epoch 6/12
37500/37500 [==============================] - 87s 2ms/step - loss: 0.0602 - acc: 0.5346 - val_loss: 0.0623 - val_acc: 0.5538
Epoch 7/12
37500/37500 [==============================] - 87s 2ms/step - loss: 0.0602 - acc: 0.5365 - val_loss: 0.0624 - val_ac

In [42]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
train_history = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_split = 0.25, epochs = 12, batch_size = 128, verbose = 1)
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 109s 3ms/step - loss: 0.3253 - acc: 0.9000 - val_loss: 0.3224 - val_acc: 0.9000
Epoch 2/12
37500/37500 [==============================] - 92s 2ms/step - loss: 0.3207 - acc: 0.9000 - val_loss: 0.3195 - val_acc: 0.9000
Epoch 3/12
37500/37500 [==============================] - 93s 2ms/step - loss: 0.3162 - acc: 0.9000 - val_loss: 0.3155 - val_acc: 0.9000
Epoch 4/12
37500/37500 [==============================] - 92s 2ms/step - loss: 0.3100 - acc: 0.9000 - val_loss: 0.3103 - val_acc: 0.9000
Epoch 5/12
37500/37500 [==============================] - 92s 2ms/step - loss: 0.3031 - acc: 0.9000 - val_loss: 0.3046 - val_acc: 0.9000
Epoch 6/12
37500/37500 [==============================] - 96s 3ms/step - loss: 0.2972 - acc: 0.9001 - val_loss: 0.2997 - val_acc: 0.9000
Epoch 7/12
37500/37500 [==============================] - 88s 2ms/step - loss: 0.2933 - acc: 0.9002 - val_loss: 0.2965 - val_ac